In [1]:
from microspike.criterion import get_pattern_repetition_number_and_spike_statistics, check_neuron_status
from microspike.utils import investigate_potential, generate_data_train_model, save_all_data, load_all_data

import ipywidgets as widgets
from IPython.display import display

- Here the neuron numbers start from 1 to N
- Pattern numbers start from 1 to P
- neuron indices start from 0 to N-1
- pattern indices start from 0 to P-1


## Generate data and train the model 

In [2]:
# SRM Hyperparamters
inhibitory_connection=True
threshold = 500
reset = 0
refractory = 1
alpha = 0.25
k = 1
tau_m = 0.010
tau_s = 0.0025
K1 = 2
K2 = 4
window_time= 7 * 0.010, # Maximum time [second] to ignore inputs before it by kernels 
train_mode= True

# Synapse Hyperparameters
approximate = True
w_max = 1
w_min = 0
A_pre = 0.03125
B = 0.85
A_post = - A_pre * B
tau_pre=0.0168
tau_post=0.0337

print('B factor is:')
print(-A_post*tau_post/(A_pre*tau_pre))


P = 3
N = 9
M = 2000
time = 450 * 1000


B factor is:
1.7050595238095239


## Define criterion

In [3]:
# input_train, model, synapse, monitor, times, indices, position_copypaste, patterns_info, timing_pattern = generate_data_train_model(P, N, M, time, w_max, w_min, A_pre, A_post, tau_pre, tau_post, approximate,
#                               threshold, reset, refractory, alpha, k, tau_m, tau_s, K1, K2, window_time,
#                               inhibitory_connection)
# save_all_data(input_train, model, synapse, monitor, times, indices,
#               position_copypaste, patterns_info, timing_pattern, 'simulation_data.pkl')

loaded_input_train, loaded_model, loaded_synapse, loaded_monitor, \
loaded_times, loaded_indices, loaded_position_copypaste, loaded_patterns_info, \
loaded_timing_pattern = load_all_data('simulation_data.pkl')


hit_rate_threshold = 0.90
false_alarm_rate_threshold = 1 # Hz

pattern_duration = 0.050 # second
inference_starting_time = 400 # second
inference_ending_time = 450 # second

pattern_repetition_number, criterion_info = get_pattern_repetition_number_and_spike_statistics(loaded_monitor, loaded_position_copypaste, inference_starting_time, inference_ending_time, pattern_duration, P)
neurons_stats, pattern_stats = check_neuron_status(N, criterion_info, pattern_repetition_number, hit_rate_threshold, false_alarm_rate_threshold, inference_starting_time, inference_ending_time)




In [5]:
# dt = 0.001
# investigate_potential(loaded_monitor, dt, loaded_position_copypaste)

In [6]:
for pattern_number, num_repetition in pattern_repetition_number.items():
    print(f'{pattern_number}: {num_repetition}')
print('-'*10)
for i in range(1, N+1):
    info = [i for i in criterion_info[f'neuron_{i}'].items()]
    print(f'neuron {i}:', info)

no_pattern: 668
pattern_1: 107
pattern_2: 114
pattern_3: 111
----------
neuron 1: [(2, 114)]
neuron 2: [('no_pattern', 2)]
neuron 3: [(3, 16)]
neuron 4: [(3, 111)]
neuron 5: [(1, 4)]
neuron 6: [(1, 107)]
neuron 7: [(3, 95)]
neuron 8: [(2, 114)]
neuron 9: [(1, 103)]


In [7]:
neurons_stats

{'neuron_1': NeuronStats(learned_pattern=2, hit_rate=1.0, false_alarm_rate=0.0, status='successful'),
 'neuron_2': NeuronStats(learned_pattern=None, hit_rate=None, false_alarm_rate=None, status='dead'),
 'neuron_3': NeuronStats(learned_pattern=3, hit_rate=0.14414414414414414, false_alarm_rate=0.0, status='unsuccessful'),
 'neuron_4': NeuronStats(learned_pattern=3, hit_rate=1.0, false_alarm_rate=0.0, status='successful'),
 'neuron_5': NeuronStats(learned_pattern=1, hit_rate=0.037383177570093455, false_alarm_rate=0.0, status='unsuccessful'),
 'neuron_6': NeuronStats(learned_pattern=1, hit_rate=1.0, false_alarm_rate=0.0, status='successful'),
 'neuron_7': NeuronStats(learned_pattern=3, hit_rate=0.8558558558558559, false_alarm_rate=0.0, status='unsuccessful'),
 'neuron_8': NeuronStats(learned_pattern=2, hit_rate=1.0, false_alarm_rate=0.0, status='successful'),
 'neuron_9': NeuronStats(learned_pattern=1, hit_rate=0.9626168224299065, false_alarm_rate=0.0, status='successful')}

In [8]:
pattern_stats

{'pattern_2': 2, 'pattern_3': 1, 'pattern_1': 2}

### Criteria 

- successful: hit rate over 90%, false alarm rate below 1 Hz (false alarm less than 1 per second)
- unsuccessful: hit rate less than 90% or false alarm rate higher than 1 Hz
- dead: general spike rate less than 1 Hz

- if the training of neuron is successful
- if the training of the entire network is successful
    - each pattern is learned by at least one neuron (hit rate over 90%, false alarm less than 1 per second)
    for each pattern, we have at least one successful neuron, then the entire network is successful.
    - what happens when N > P, P is fixed and N is increasing. N = [100, 75, 50, 25, 10,]

        for N = 50 --> P = [0.1, 0.25, 0.5, 0.75, 1] * N
        
                        [more dead neurons, ...., less dead neurons]
        
        for N = 100 --> P = [0.1, 0.25, 0.5, 0.75, 1] * N
        
                        [more dead neurons, ...., less dead neurons]    
    - for each pattern how many neurons have learned the pattern
    - record the number of dead neurons, per each set of hyperparameters


- Reward modulated. If it can increase the number of successful neurons 
    - digit recognition task MNIST dataset using anti STDP (reverse).
    - objective is to have the same number of neurons and patterns. (no dead neurons and no neurons responding to the same pattern). one pattern for one neuron only.
    - we give negative reward when a neuron fires during another pattern. Anti STDP.
    - we do normal STDP when the neuron fires during its own pattern and during no-pattern.
    - we do not implement inhibitory connections and replace their role with R-STDP. 
    - Also do R-STDP and inhibitory connections together and compare the results.

    - based on the results we can modify the rule. 

- introduce a new pattern to the network during inference time.
    - do this the last. use the results of the previous ones and then plan this one. we can use R-STDP in this step too.